In [ ]:
import psutil
import humanize
import os
from IPython.display import display_html
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

print(os.listdir("../input"))
dataDirectory= "./input/plantdisease-upd1"
print(os.listdir(dataDirectory))



In [ ]:
import numpy as np
import keras
import cv2
from time import time
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder  
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imutils import paths
import itertools
import matplotlib.pyplot as plt
import time
import pandas as pd
%matplotlib inline

In [ ]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images("/kaggle/input/plantdisease-upd1"))

In [ ]:
Classes_names=['Pepper_Bacterial', 'Potato_healthy', 'Tomato_Leaf_Mold', 'Tomato_YellowLeaf_Virus', 'Tomato_Bacterial_spot', 'Tomato_Septoria_spot', 'Tomato_healthy', 'Tomato_spider_mite', 'Tomato_Early_blight', 'Tomato_Target_Spot', 'Pepper_healthy', 'Potato_Late_blight', 'Tomato_Late_blight', 'Potato_Early_blight', 'Tomato_mosaic_virus']

In [ ]:
images_path = dataDirectory+'/PlantVillage_upd1'
images_datagen = ImageDataGenerator()            # used for image transformations in the current scenario used for plotting

In [ ]:
img_generator = images_datagen.flow_from_directory(
    images_path ,
    color_mode="rgb",
    target_size=(224,224),
    )

In [ ]:
 # Defining a function for customizable plot
def plots(ims, figsize = (22,22), rows=4, interp=False, titles=None, maxNum = 9):
    if type(ims[0] is np.ndarray):
        ims = np.array(ims).astype(np.uint8)
        if(ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
           
    f = plt.figure(figsize=figsize)
    #cols = len(ims) //rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    cols = maxNum // rows if maxNum % 2 == 0 else maxNum//rows + 1
    #for i in range(len(ims)):
    for i in range(maxNum):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=20)
        plt.imshow(ims[i], interpolation = None if interp else 'none')  

In [ ]:
imgs, labels = img_generator.next()
labelNames=[]
labelIndices=[np.where(r==1)[0][0] for r in labels]
#print(labelIndices)

for ind in labelIndices:
    for labelName,labelIndex in img_generator.class_indices.items():
        if labelIndex == ind:
            #print (labelName)
            labelNames.append(labelName)


In [ ]:
plots(imgs, rows=6, titles = labelNames, maxNum=15)

In [ ]:
# Importing Resnet50 model from keras for feature extraction
from keras.models import Sequential
from keras.models import Model
model=Sequential()   #creating our model to obtain features
Resnet_base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224,3))  #removing top layer
Resnet_base_model.summary()
model.add(Resnet_base_model)
model.add(Flatten())   # obtaining a feature vector along a single axis
model.summary()
#Resnet_base_model.summary()

In [ ]:
feature_matrix=[]  # declaring an empty array for feature storage
labels = []       #  declaring labels array 

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
           label = imagePath.split(os.path.sep)[-2]   # obtain a label from the class folder 
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
           image = cv2.imread(imagePath)
           image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
           image = cv2.resize(image, (224, 224))
           image = image.reshape(1,224,224,3) # update the data and labels lists, respectively
           image = preprocess_input(image)      #convert to a scale that model uses    
           features = model.predict(image)
           feature_matrix.append(features)
           labels.append(label)
          
# converting the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]   i-e normalization
feature_matrix=np.array(feature_matrix)/255     
labels=np.array(labels)


In [ ]:
le = LabelEncoder()            #encode labels
kk=le.fit_transform(labels)    #Assign integer labels
Classes=le.classes_            #number of unique classes
labels = to_categorical(kk)    # assign binary numeric array to labels
labels

In [ ]:
#Obtaining training (80%) and testing (20%) data using holdout

(trainX, testX, trainY, testY) = train_test_split(feature_matrix, labels,
    test_size=0.20, stratify=labels, random_state=0)
print('Train size: {train}, Test size: {test}'.format(train=trainX.shape[0], test=testX.shape[0]))

In [ ]:
trainX=trainX[:,0,]     #get the desired shape (number of samples, size of feature vector) (i-e 2662,100352)
print(trainX.shape)
testX=testX[:,0,]     #get the desired shape (number of samples, size of feature vector) (i-e 666,100352)
print(testX.shape)


In [ ]:
np.random.seed(0)         
INPUT_LENGTH = 100352    #FEATURE VECTOR DIMENSION
HIDDEN_UNITS = 1000       #NUMBER OF NEURONS

Win = np.random.normal(size=[INPUT_LENGTH, HIDDEN_UNITS])     #RANDOM INITIALIZATION OF WEIGHTS
print('Input Weight shape: {shape}'.format(shape=Win.shape))

In [ ]:
def input_to_hidden(x):    # defining function for the dot product b/w input and its weights 
    a = np.dot(x, Win)
    a = np.maximum(a, 0, a) # Implementation of ReLU activation function
    return a

In [ ]:
def predict(x):
    x = input_to_hidden(x)
    y = np.dot(x, Wout)
    return y

In [ ]:
    #Training process of ELM model
    start = time.time()
    X = input_to_hidden(trainX)
    Xt = np.transpose(X)
    Wout = np.dot(np.linalg.inv(np.dot(Xt, X)), np.dot(Xt, trainY))
    print('Output weights shape: {shape}'.format(shape=Wout.shape))
    print ('Total Training Time is: %s seconds ' % (time.time()-start))

In [ ]:
y = predict(testX)        #predict test data
gr_labs = []              #ground truths
pred_labs = []            # predicted labels
correct = 0
total = y.shape[0]
for i in range(total):
        predicted = np.argmax(y[i])
        test = np.argmax(testY[i])
        #print(test)
        gr_labs.append(test)
        pred_labs.append(predicted)
        correct = correct + (1 if predicted == test else 0)
        Acc=correct/total
print('Accuracy: {:f}'.format(correct/total))

In [ ]:
print(classification_report(gr_labs,  pred_labs, target_names=Classes_names))   

In [ ]:
matrix = confusion_matrix(gr_labs,  pred_labs)

In [ ]:
#Prepared code that is taken from SKLearn Website, Creates Confusion Matrix
def plot_confusion_matrix(cm, Classes_names,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.rcParams.update({'font.size': 16})
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(Classes_names))
    plt.xticks(tick_marks, Classes_names, rotation=45,fontsize=18,weight = 'bold')
    plt.yticks(tick_marks, Classes_names,fontsize=18,weight = 'bold')

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 fontsize=16,
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=20, weight = 'bold')
    plt.xlabel('Predicted label',fontsize=20, weight = 'bold')

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(20,20), dpi=200)
cm_plot_labels = Classes_names
plot_confusion_matrix(matrix,cm_plot_labels, normalize=False
                      , title = 'Confusion Matrix')
